In [ ]:
import os
import shutil
import csv

# creo liste di appoggio
SubDir_list = os.listdir("files/") # lista sottocartelle e file della cartella files
ImagesExtensions = ['.png', '.jpg', '.jpeg', '.gif'] # lista estensioni immagini
DocumentsExtensions = ['.doc', '.docx', '.odt', '.pages', '.txt'] # lista estensioni documenti
AudioExtensions = ['.mp3', '.acc', '.wav'] # lista estensioni audio


# se il file non esiste lo creo e scrivo l'intestazione
if not os.path.isfile(os.path.join("files", "recap.csv")): # verifico esistenza file
    with open(os.path.join("files", "recap.csv"), 'a') as recap:
        writer = csv.writer(recap)
        writer.writerow(["name", "type", "size"]) # inserisco intestazione
            
# apro file recap.csv
with open(os.path.join("files", "recap.csv"), 'a') as recap:
    
    # ciclo files per spostare i file in sottocartelle
    for f in sorted(SubDir_list): # analizzo files nella cartella files
        name, ext = os.path.splitext(f) # ricavo nome ed estensione file
        size = os.path.getsize(os.path.join("files", f)) # trovo dimensione file

        # assegno nome sottocartella e gestico possibili files non riconosciuti
        if ext in ImagesExtensions:
            SubDirName = 'Images'
        elif ext in DocumentsExtensions:
            SubDirName = 'Docs'
        elif ext in AudioExtensions:
            SubDirName = 'Audio'
        else:
            if ext == "" or f == "recap.csv": # gestisco file di stistema nascosti, cartelle ed il file recap.csv
                continue 
            else: # gestisco possibili estensioni non supportate comunicandolo all'utente
                print(f"il file {f} non può essere spostato in una sottocartella perchè l'estensione {ext} non è supportata")
                continue

        # se non già presente aggiungo nome sotto-cartella alla lista e creo sotto-cartella
        if SubDirName not in SubDir_list:
            SubDir_list += [SubDirName] # aggiungo il nome della cartella alla lista
            os.makedirs(os.path.join("files", SubDirName)) # creo sotto-cartella 

        # sposto il file nella sottocartella
        shutil.move(os.path.join("files", f), os.path.join("files", SubDirName))

        # stampo info richieste
        print(f"name: {name}, type: {SubDirName}, size: {size}") 

        # aggiorno il file di racap
        writer = csv.writer(recap)
        writer.writerow([name, SubDirName, size])


In [ ]:
import os
import numpy as np
from PIL import Image
from PIL import UnidentifiedImageError
from tabulate import tabulate

info = [] # lista per tutte le info delle immagini su cui strutturare tabella

# ciclo gli elementi della cartella Images
for i in os.listdir("files/Images"):
    
    try:
        img = Image.open(os.path.join("files/Images", i)) # apro l'immagine
        ImgArray = np.asarray(img) # array dell'immagine

        # creo i dati utili alla tabella
        img_name, img_ext = os.path.splitext(i) # nome
        img_h, img_w = np.shape(ImgArray)[0:2] # altezza e larghezza in pixel
        img_shape = np.shape(ImgArray) # shape img
        img_ndim = ImgArray.ndim # numero dimensioni

        # inizializzo i valori delle medie dei colori ad ogni ciclo
        P, R, G, B, A = 0, 0, 0, 0, 0

        # ricavo il tipo di immagine GrayScale, RGB o RGBA ed assegno le medie dei valori dei colori
        if img_ndim == 2: # immagine B&W, scala di grigi o Palette
            P = np.mean(ImgArray)
        else:  # img_ndim == 3:
            if img_shape[2] == 3: # immagine RGB
                R, G, B = np.mean(ImgArray, axis=(0,1))
            elif img_shape[2] == 4: # immagine RGBA
                R, G, B, A = np.mean(ImgArray, axis=(0,1))
            else:  # altro tipo di immagine non supportato
                print(f"il file {i} non è supportato")

        # creo dict con i dati della tabella
        data = {"Name": img_name, 
                "Height": img_h, 
                "Width": img_w, 
                "Palette/B&W": P, 
                "Red": R, 
                "Green": G, 
                "Blue": B,  
                "Alpha": A}

        info += [data] # aggiorno lista tabella con dict con dati immagine

    except UnidentifiedImageError:
        print(f"""
        il file {i} non è riconosciuto come un'immagine:
        non è possibile inserire i suoi dati nella tabella
        """)
    except IsADirectoryError:
        print(f"""
        {i} è una cartella:
        non è possibile inserire i suoi dati nella tabella""")

# stampo tabella
print(tabulate(info, headers='keys', tablefmt='fancy_grid', floatfmt=".2f"))
